# Genetic Algorithms

![image.png](https://upload.wikimedia.org/wikipedia/commons/thumb/c/c3/Metaheuristics_classification.svg/630px-Metaheuristics_classification.svg.png)

First, we setup the genetic representation of a solution

In [1]:
from typing import List
from random import choices

Genome = List[int]

def generate_genome(length: int) -> Genome:
    return choices([0, 1], k=length)

We need also to generate new solutions, i.e. population

In [2]:
Population = List[Genome]

def generate_population(pop_size: int, genome_length: int) -> Population:
    return [generate_genome(genome_length) for _ in range(pop_size)]

Let's define the fitness function that'll be used in the evaluation phase

In [3]:
from collections import namedtuple

# part of the solution
Individual = namedtuple("Individual", ["name", "value", "weight"])

def fitness(genome: Genome, individuals: [Individual], weight_limit: int) -> int:
    if len(genome) != len(individuals):
        raise ValueError("Genome and individuals must have the same length")

    weight = 0
    value = 0

    for i, individual in enumerate(individuals):
        if genome[i] == 1:
            weight += individual.weight
            value += individual.value

            if weight > weight_limit:
                return 0
    
    return value

Define the selection function to select pair of candidate solutions for the next generations

In [4]:
from typing import Callable

FitnessFunc = Callable[[Genome], int]

def selection_pair(population: Population, fitness_func: FitnessFunc):
    return choices(
        population=population,
        weights=[fitness_func(genome) for genome in population],
        k=2
    )

Now, we define the single-point crossover function

In [5]:
from typing import Tuple
from random import randint

def single_point_crossover(a: Genome, b: Genome) -> Tuple[Genome, Genome]:
    # handle edge cases
    if len(a) != len(b):
        raise ValueError("Genomes should have the same length to apply the crossover")

    length = len(a)
    if length < 2:
        return a, b

    # randomly set the point at which the crossover will occur
    p = randint(1, length - 1)
    
    # return the children
    return a[0:p] + b[p:], b[0:p] + a[p:]

Lastly, we need the mutation function

In [6]:
from random import randrange, random


def mutate(genome: Genome, num: int = 1, probability: float = 0.5) -> Genome:
    # by default, 2 genes are mutated
    for _ in range(num):
        index = randrange(len(genome))

        # flips the gene value (e.g. changes 0 to 1 and vice versa)
        genome[index] = genome[index] if random() > probability else abs(genome[index] - 1)
    return genome

In [7]:
PopulateFunc = Callable[[], Population]
SelectionFunc = Callable[[Population, FitnessFunc], Tuple[Genome, Genome]]
CrossoverFunc = Callable[[Genome, Genome], Tuple[Genome, Genome]]
MutationFunc = Callable[[Genome], Genome]

def run_evolution(
    populate_func: PopulateFunc,
    fitness_func: FitnessFunc,
    fitness_limit: int,
    selection_func: SelectionFunc = selection_pair,
    crossover_func: CrossoverFunc = single_point_crossover,
    mutation_func: MutationFunc = mutate,
    generation_limit: int = 100,


) -> Tuple[Population, int]:
    # generate the population
    population = populate_func()

    # iterate over generations 
    for i in range(generation_limit):
        
        # sort population based on the solutions fitness
        population = sorted(
            population,
            key=lambda genome: fitness_func(genome),
            reverse=True
        )

        # return early if we reach the fitness limit (hint: we can implement elitism here)
        if fitness_func(population[0]) >= fitness_limit:
            break
            
        # keep the top 2 solutions for the next generation
        next_generation = population[0:2]

        # generate the solutions for the next generation
        for j in range(int(len(population) / 2) - 1):
            parents = selection_func(population, fitness_func)
            offspring_a, offspring_b = crossover_func(parents[0], parents[1])
            
            offspring_a = mutation_func(offspring_a)
            offspring_b = mutation_func(offspring_b)

            next_generation += [offspring_a, offspring_b]
        
        population = next_generation

    # check if we 
    population = sorted(
        population,
        key=lambda genome: fitness_func(genome),
        reverse=True
    )

    return population, i
                

Now, it's time to run the algorithm. We'll solve the knapsack problem to demonstrate how the problem runs, given that the weight limit of knapsack is 3000.

In [47]:


items = [
    Individual('Laptop', 500, 2200),
    Individual('Headphones', 150, 160),
    Individual('Coffee Mug', 60, 350),
    Individual('Notepad', 40, 333),
    Individual('Water Bottle', 30, 192),
]

more_items = [
    Individual('Mints', 5, 25),
    Individual('Socks', 10, 38),
    Individual('Tissues', 15, 80),
    Individual('Phone', 500, 200),
    Individual('Baseball Cap', 100, 70)
] + items


# helper function to convert the genome representation of solution to the oiginal one
def genome_to_items(genome: Genome, items: [Individual]) -> [Individual]:
    result = []
    for i, item in enumerate(items):
        if genome[i] == 1:
            result += [item.name]
    return result


from functools import partial


# functions were made to be generalized and abstract
# so we create a custom version using functools.partial so they meet our needs in knapsack problem

population, generations = run_evolution(
    populate_func=partial(
        generate_population,
        pop_size=10,
        genome_length=len(items)
    ),
    fitness_func=partial(
        fitness, 
        individuals=items,
        weight_limit=3000
    ),
    fitness_limit=740,
    generation_limit=100
)



print(f"Number of generations: {generations}")
print(f"Best solution: {genome_to_items(population[0], items)}")

Number of generations: 0
Best solution: ['Laptop', 'Headphones', 'Coffee Mug', 'Water Bottle']


# TODOs
- Plot a chart showing the difference in fitness over generations
- Plot the distribution of many algorithm runs

# References

- https://en.wikipedia.org/wiki/Metaheuristic
- https://github.com/kiecodes/genetic-algorithms    
    - https://www.youtube.com/watch?v=nhT56blfRpE